This script is EDA for scraping NYC zoning and regulation data from the planning.nyc.gov website. In the end, I'd like to create a research tool for NYC zoning and regulation and perhaps a search engine to help developers and policy makers find the appropriate piece of legistation.

For now, we can create a structure table to collect all the data, but then we are going to have to create some type of strucutre to process the data in a meaningful way. For example, for one article there might be multiple chapters and in those chapters certain itmes might not be broken down but instead in one large text. This type of text data needs to be organize and filtered somehow.

In [74]:
# Libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

# Regex remove new line breaks when getting text
regex = re.compile(r'\n', re.IGNORECASE)

# URL link for NYC zoning
url = 'https://zr.planning.nyc.gov'
response = requests.get(url)

# parameters
article_parameter = '/article-'

columns = ['State', 'City','Article_Number', 'Article_Link', 'Article_Text', 'Chapter_Number', 'Chapter_Link', 'Chapter_Text']

df = pd.DataFrame(columns = columns)

# BS
soup = BeautifulSoup(response.text, "html.parser")

# Finds all article tags in the HTML
articleHTML = soup.findAll('article')

# Loops to collect all the article links
for i in range(len(articleHTML)):
    
    # Find all articles
    one_article_tag = soup.findAll('a')[i]
    
    # Find all href articles
    link = one_article_tag['href']
    
    # Make sure we are only collecting article related data
    if(link.find(article_parameter) >= 0):
        
        # chapter parameter
        chapter_parameter =  link + '/chapter-'

        # get article text
        article_text = one_article_tag.find('div', attrs = {'class': 'field field--name-field-article-title field--type-string field--label-hidden field__item'})

        #  find the chapters
        url_article =  url + link
        response_chapter = requests.get(url_article)
        soup_chapter = BeautifulSoup(response_chapter.text, "html.parser")
        chapterHTML = soup_chapter.find('div', attrs = {'class': 'article-chapters'}).find('div', attrs = {'class': 'view-content'}).findAll('div', attrs = {'class': 'views-row'})
        
        # traverse through chapters
        for j in range(len(chapterHTML)):
            
            # list of chapters
            one_chapter_tag = chapterHTML[j]
            
            # chapter link at position 0 of the "a" tag
            link_chapter = one_chapter_tag.findAll('a')[0]['href']
            
            # Chapter text is the next sibling after span
            chapter_text = one_chapter_tag.find('span', attrs = {'class': 'field field--name-title field--type-string field--label-hidden'}).next_sibling
            
            # Replace new lines ('\n') with blank spaces
            chapter_text = re.sub(regex, '', chapter_text)
        
            if(link_chapter.find(chapter_parameter) >= 0):
            
                # populate table with article and chapters
                df = df.append({
                'Article_Number': link,
                'Article_Link': url_article,
                'Article_Text': article_text.text,
                'Chapter_Number': link_chapter,
                'Chapter_Link': url_article + link_chapter,
                'Chapter_Text': chapter_text
            }, ignore_index = True)

# Adds state and city column
df['State'] = 'NY'
df['City'] = 'NYC'


 - Title, Establishment of Controls and Interpretation of Regulations
 - Construction of Language and Definitions
 - Comprehensive Off-street Parking and Loading Regulations in the Manhattan Core
 - Sidewalk Cafe Regulations
 - Residential Conversion within Existing Buildings
 - Comprehensive Off-street Parking Regulations in the Long Island City Area
 - Statement of Legislative Intent
 - Use Regulations
 - Residential Bulk Regulations in Residence Districts
 - Bulk Regulations for Community Facilities in Residence Districts 
 - Accessory Off-Street Parking and Loading Regulations
 - Special Urban Design Regulations
 - The Quality Housing Program
 - Statement of Legislative Intent
 - Use Regulations
 - Bulk Regulations for Commercial or Community Facility Buildings in Commercial Districts
 - Bulk Regulations for Residential Buildings in Commercial Districts 
 - Bulk Regulations for Mixed Buildings in Commercial Districts
 - Accessory Off-Street Parking and Loading Regulations
 - Specia

In [75]:
df.head()

,State,City,Article_Number,Article_Link,Article_Text,Chapter_Number,Chapter_Link,Chapter_Text
0,NY,NYC,/article-i,https://zr.planning.nyc.gov/article-i,General Provisions,/article-i/chapter-1,https://zr.planning.nyc.gov/article-i/article-...,"- Title, Establishment of Controls and Interp..."
1,NY,NYC,/article-i,https://zr.planning.nyc.gov/article-i,General Provisions,/article-i/chapter-2,https://zr.planning.nyc.gov/article-i/article-...,- Construction of Language and Definitions
2,NY,NYC,/article-i,https://zr.planning.nyc.gov/article-i,General Provisions,/article-i/chapter-3,https://zr.planning.nyc.gov/article-i/article-...,- Comprehensive Off-street Parking and Loadin...
3,NY,NYC,/article-i,https://zr.planning.nyc.gov/article-i,General Provisions,/article-i/chapter-4,https://zr.planning.nyc.gov/article-i/article-...,- Sidewalk Cafe Regulations
4,NY,NYC,/article-i,https://zr.planning.nyc.gov/article-i,General Provisions,/article-i/chapter-5,https://zr.planning.nyc.gov/article-i/article-...,- Residential Conversion within Existing Buil...


In [57]:
# workingon chapters
url = 'https://zr.planning.nyc.gov/article-i'

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

test = soup.find('div', attrs = {'class': 'article-chapters'}).find('div', attrs = {'class': 'view-content'}).findAll('div', attrs = {'class': 'views-row'})

len(test)
#test2 = test[1]
#test2.findAll('a')[0]['href']

6

In [62]:
regex = re.compile(r'/n', re.IGNORECASE)
link = '/article-i'
url_article =  url + link
response_chapter = requests.get(url_article)
soup_chapter = BeautifulSoup(response_chapter.text, "html.parser")
# need to fix the soup_chapter.find
chapterHTML = soup_chapter.find('div', attrs = {'class': 'article-chapters'}).find('div', attrs = {'class': 'view-content'}).findAll('div', attrs = {'class': 'views-row'})

chapterHTML = chapterHTML[5]

chapter_text = chapterHTML.find('span', attrs = {'class': 'field field--name-title field--type-string field--label-hidden'}).next_sibling

chapter_text = re.sub(regex, '', chapter_text)
print(chapter_text)

chapterHTML


 - Comprehensive Off-street Parking Regulations in the Long Island City Area


<div class="views-row">
<article about="/article-i/chapter-6" class="node node--type-chapter node--view-mode-leftnav" data-history-node-id="85" role="article">
<a class="chapter-title" href="/article-i/chapter-6"><span class="field field--name-title field--type-string field--label-hidden">Chapter 6</span>
 - Comprehensive Off-street Parking Regulations in the Long Island City Area</a>
</article>
</div>